In [6]:
!pip install git+https://github.com/twintproject/twint
!pip install aiohttp_socks

  Cloning https://github.com/twintproject/twint to /private/var/folders/8q/h7x35g6s20l48s4yxqvzw1w40000gp/T/pip-req-build-1sfehml9
  Running setup.py bdist_wheel for twint ... done
  Stored in directory: /private/var/folders/8q/h7x35g6s20l48s4yxqvzw1w40000gp/T/pip-ephem-wheel-cache-ec49en82/wheels/44/02/e0/86b03deb4fb12efcb200d22b87efb9b38ee14ed6bfaacd02d2
Successfully built twint


In [7]:
#!pip install -q tweepy
%reload_ext autoreload
%autoreload 2
import twint
import logging
import pandas as pd
import datetime
import tweepy
import json
from utils.extract import db_connection, download_data
logging.getLogger().setLevel(logging.WARNING)

In [8]:
#credenciales de db, twitter y emisoras
with open('creds.txt', encoding='utf-8') as data_file:
        creds = json.loads(data_file.read())

with open('tw_creds.txt', encoding='utf-8') as data_file:
    tw_creds = json.loads(data_file.read())

lista_cuentas = pd.read_csv('Cuentas BMBV.csv')



In [9]:
c = twint.Config()
c.Username = "BWcapitalMex"
c.Limit = 10
c.Pandas = True

In [ ]:
twint.run.Search(c)

In [11]:
tweets_df = twint.storage.panda.Tweets_df

In [ ]:
tweets_df.head()

In [13]:
def obtiene_todos_tweets(cuenta):
    
    c = twint.Config()
    c.Username = cuenta.replace('@','') # por si metemos la cuenta con @
    c.Pandas = True
  
    twint.run.Search(c)
  
    df = twint.storage.panda.Tweets_df
  
    return df

def obtiene_todo_contenido(contenido, limit=1000):
    
    c = twint.Config()
    c.Search = contenido
    c.Pandas = True
    c.Limit = limit
  
    twint.run.Search(c)
  
    df = twint.storage.panda.Tweets_df
  
    return df
  
def update_in_bulk_db(creds):
    """
    Añade todos los tweets de una vez
    """
    query_table = """
            CREATE TABLE IF NOT EXISTS
                tweets (
                    id integer not null,
                    user_id integer,
                    date text not null,
                    time text not null,
                    timezone text not null,
                    location text not null,
                    user text not null,
                    tweet text not null,
                    replies integer,
                    likes integer,
                    retweets integer,
                    hashtags text,
                    link text,
                    retweet bool,
                    user_rt text,
                    mentions text,
                    date_update text not null,
                    PRIMARY KEY (id)
                );
        """
    conn = db_connection(creds)
    
    try:
        download_data(conn,query_table)
    except Exception as e:
        print('Error al crear tabla de tweets'.format(e))
    
    
    
    try:
      
      for cuenta in lista_cuentas['Cuentas']:
        df = obtiene_todos_tweets(cuenta)
        
        conn = db_connection(creds)
        date_time = str(datetime.now())
      
        lista_tweets = df.values.tolist()
        lista_tweets = [tuple(tweet.append(date_time)) for tweet in lista_tweets]
        
    
        print('se van a guardar {} tweets de la cuenta: {}'.format(len(lista_tweets), lista_tweets[0][12]))
    
        data_ready = str(lista_tweets[1:-1])
    
        query= """INSERT INTO tweets VALUES {};""".format(data_ready)
    
        download_data(conn,query)
      
    except Exception as e:
        print('Error al insertar en la base'.format(e))


In [14]:
update_in_bulk_db(creds)

ERROR:root:Error en download data
ERROR:root:syntax error at or near "user"
LINE 10:                     user text not null,
                             ^



Error al insertar en la base
1046172818684153856 2018-09-29 18:00:11 CDT <BMVMercados> Traxion, empresa dedicada la industria del autotransporte y logística en México celebra su primer año como #EmpresaBolsa. pic.twitter.com/4ajFvGSJnE
1046127520309886976 2018-09-29 15:00:11 CDT <BMVMercados> .@elcidresorts controladora de acciones de empresas hoteleras y de esparcimiento, cumple 21 años de listado en #LaBolsaDeMéxico. #EmpresasBolsa pic.twitter.com/FuqCwpdAyl
1046082244413214726 2018-09-29 12:00:16 CDT <BMVMercados> Conoce el comportamiento del mercado en nuestro boletín semanal pic.twitter.com/aaKCvcMvSQ
1045848135845761030 2018-09-28 20:30:01 CDT <BMVMercados> Grupo México Transportes cumple un año como emisora de #LaBolsaDeMéxico. ¡Felicidades! pic.twitter.com/ZeYkacoYzr
1045840599159640064 2018-09-28 20:00:04 CDT <BMVMercados> .@ASURMexico ofrece servicios aeroportuarios, hoy celebra 18 años de listado en #LaBolsaDeMéxico. ¡Muchas felicidades!. #EmpresasBolsa pic.twitter.com/0GDHH

1044738360366047234 2018-09-25 19:00:10 CDT <BMVMercados> .@jbonham_carter, enviada especial para comercio del Reino Unido en México, sostuvo un encuentro con directivos de #LaBolsaDeMéxico y @mexico2_pmc para profundizar nuestra colaboración sobre #bonosverdes, infraestructura sustentable y mejores prácticas en #financiamientoclimatico pic.twitter.com/6jpbgLMmxY
1044684254662283264 2018-09-25 15:25:10 CDT <BMVMercados> Cierre de Mercado: $BMV S&P/BMV IPC 49,651.55 / +0.47% Dólar Spot: 18.9970
1044683519371423750 2018-09-25 15:22:14 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS S&P/BMV IPC DC18= 50,440 / +323.00 / -0.64% USD/MX DC18= $19.2560 / +0.05 / +0.25%
1044670416881700864 2018-09-25 14:30:11 CDT <BMVMercados> Hoy se conocieron los datos del IGAE en nuestro país, ¿Cuál fue la reacción de los inversionistas? hoy en #DesdeLaBolsa Carlos González Tabares Director de Análisis y Estrategia Bursátil Monex14:40hrs FOROtv
1044608285742174211 2018-09-25 10:23:17 CDT <BMVMercad

1043213466390147083 2018-09-21 14:00:47 CDT <BMVMercados> .@FitchRatings le asignó a la Bolsa Mexicana de valores, que cotiza bajo la clave “BOLSA”, las calificaciones en escala internacional (IDRs) de largo plazo en moneda extranjera y local de 'BBB+' y de largo plazo en escala nacional 'AAA(mex)', con Perspectiva Estable. pic.twitter.com/mNRODUrr37
1043135848663994369 2018-09-21 08:52:21 CDT <BMVMercados> Apertura de Mercado: $BMV S&P/BMV IPC 49,436.63 / -0.15% Dólar Spot: 18.8885
1043135818066526209 2018-09-21 08:52:14 CDT <BMVMercados> $MexDer Apertura $FUTUROS S&P/BMV IPC DC18= 50,405 / +112.00 / +0.22% USD/MX DC18= $19.1523 / +0.08 / +0.44%
1042952308076765184 2018-09-20 20:43:02 CDT <BMVMercados> Estamos impulsando la Educación Financiera con el lanzamiento del blog #HablemosDeBolsa y el rediseño de la #EscuelaBolsaMexicana de la que han egresado más de 20 mil estudiantes y hemos recibido a 25 mil personas que visitan la sede de #LaBolsaDeMéxico pic.twitter.com/egXUZAr8ES
104295

1041685708266713088 2018-09-17 08:50:01 CDT <BMVMercados> $MexDer Apertura $FUTUROS S&P/BMV IPC SP18= 49,720 / -77.00 / -0.15% USD/MX SP18= $18.8384 / -0.04 / -0.19%
1041371159672115202 2018-09-16 12:00:06 CDT <BMVMercados> En #LaBolsaDeMéxico celebramos el orgullo de ser mexicanos. ¡Felices Fiestas Patrias! pic.twitter.com/LLXrKse3wN
1040701062578151424 2018-09-14 15:37:23 CDT <BMVMercados> Cierre de Mercado: $BMV S&P/BMV IPC 49,611.93 / -0.16% Dólar Spot: 18.8623
1040700696000167937 2018-09-14 15:35:55 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS S&P/BMV IPC SP18= 49,797 / -56.00 / -0.11% USD/MX SP18= $18.8748 / +0.01 / +0.03%
1040681590882480129 2018-09-14 14:20:00 CDT <BMVMercados> Factores podrían influenciar en el tipo de cambio hacia la siguiente semana hoy en #DesdeLaBolsa Fernando Bolaños Analista Bursátil Monex 14:40hrs FOROtv
1040657575048564736 2018-09-14 12:44:35 CDT <BMVMercados> Seguramente has escuchado hablar acerca de los Derivados Financieros, pero proba

...

22110372958 2010-08-25 13:41:09 CDT <BMVMercados> BMV: -30pts, -0.10% /Dow: +27pts, +0.27% // Dólar 12.98// La venta de casas nuevas en EU se desplomaron 12.4% (mínimo histórico)// ER//13:40
22089394956 2010-08-25 08:44:48 CDT <BMVMercados> BMV: -210pts, -0.67% /Dow: -36pts, -0.36% // Dólar 13.05// Las Órdenes de Bienes Duraderos en EU subieron 0.3% en julio (2.8%e) // ER//08:45
22028201744 2010-08-24 15:20:54 CDT <BMVMercados> BMV: -777pts, -2.42% /Dow: -134pts, -1.32% // Dólar 12.95 // Emisoras Operadas 86 // Alzas 09 // Bajas 65 // Sin Cambio 12 // ER /15:20
22017599818 2010-08-24 12:26:11 CDT <BMVMercados> BMV: -547pts, -1.7% /Dow: -101pts, -0.99% // Dólar 12.95 // La venta de casas usadas en Estados Unidos bajó 27.2% en julio // ER // 12:25
22001581835 2010-08-24 08:46:03 CDT <BMVMercados> BMV: -397pts, -1.24% /Dow: -97pts, -0.96% // Dólar 12.98 // Mercados caen por mayor aversión al riesgo ante recuperación económica/ER/08:50
21939740147 2010-08-23 15:12:20 CDT <BMVMercados> BMV:

19910834208 2010-07-30 08:49:21 CDT <BMVMercados> Bolsa:-142pts, -0.44% /Dow:-74pts, -0.70% //Dólar 12.75 // Baja a 2.4% estimación del PIB en EU para el 2T10 // ER
19846704087 2010-07-29 13:40:31 CDT <BMVMercados> Bolsa:-128pts, -0.39% /Dow:-20.3pts, -0.19% //Dólar 12.70 // Crece una mayor perspectiva de deflación en EU (Fed San Luis) // ER
19827342589 2010-07-29 08:53:07 CDT <BMVMercados> Bolsa:+7pts, +0.02% /Dow:+64pts, +0.61% //Dólar 12.65 // Bajan solicitudes de desempleo en EU y sube confianza económica en zona euro// ER
19366214140 2010-07-23 15:20:48 CDT <BMVMercados> Bolsa:+85pts, +0.26% /Dow:+102pts, +0.99% //Dólar 12.73 // Resultados positivos de las instituciones sometidas al Stres Test en Europa// ER
19340195107 2010-07-23 08:44:07 CDT <BMVMercados> Bolsa:+65.5pts, +0.20% /Dow:-1.9pts, -0.02% //Dólar 12.80 // La confianza de las empresas alemanas registró un alza récord en julio // ER
19077912417 2010-07-21 08:53:15 CDT <BMVMercados> Bolsa:+227pts, +0.71% /Dow:-1.5pts, -0.

15648087044 2010-06-07 13:34:46 CDT <BMVMercados> Bolsa:-111pts,-0.36%/Dow:-26pts,-0.26%/Dólar,12.92// Mercados volátiles sin referentes económicos para hoy // ER 13:35
15629661701 2010-06-07 08:38:40 CDT <BMVMercados> Bolsa:+42pts,+0.13%/Dow:+5.5pts,+0.5%/Dólar,12.96//Suben más de lo estimado pedidos industriales en Alemania // ER 08:38
15416140282 2010-06-04 08:39:22 CDT <BMVMercados> Bolsa:-202pts,-0.65%/Dow:-155pts-1.52%/Dólar,12.84//Nominas no agricolas crecen menor a lo esperado en EU // ER 08:39
15348550400 2010-06-03 15:05:29 CDT <BMVMercados> Bolsa:-80pts,-0.25%/Dow:+6pts+0.06%/Dólar,12.79//BMV cierra a la baja por datos no satisfactorios en EU // ER 15:05
15331784399 2010-06-03 10:15:48 CDT <BMVMercados> Bolsa:+149.99pts,+0.48%/Dow:+14.51pts+0.14%/Dólar,12.73//Avanzan BMV y DJI ante datos económicos mixtos en EU // AM 10:15
15274474017 2010-06-02 15:16:35 CDT <BMVMercados> Bolsa:+166pts,+0.53%/Dow:+225pts+2.25%/Dólar,12.80// Mercados impulsados por dato favorable de ventas pe

13932608291 2010-05-13 15:19:44 CDT <BMVMercados> Bolsa: -37pts-0.11%/Dow: -114Pts-1.05%/Dólar,12.42/ Mercados a la baja . Precupa investigación a Bancos en EU //ER 15:15
13928513799 2010-05-13 13:42:20 CDT <BMVMercados> Bolsa: -10.pts-0.03%/Dow: -12Pts-0.11%/Dólar,12.36/ Mercados volátiles por investigación de la SEC a Bancos en Estados Unidos //ER 13:41
13914791566 2010-05-13 08:46:31 CDT <BMVMercados> Bolsa: -27pts-0.09%/Dow: -34Pts-0.52%/Dólar,12.37/ Solicitudes de desempleo en EU bajan menos de lo esperado (444mil vs 440mil e) //ER 08:44
13869578868 2010-05-12 15:13:18 CDT <BMVMercados> Bolsa:+260pts+0.81%/Dow:+148Pts+1.38%/Dólar,12.37/ Mercados cierran positivos por Plan Fiscal de España y Balanza Comercial en EU//ER 15:10
13866646162 2010-05-12 14:04:53 CDT <BMVMercados> Bolsa: +268pts+0.83%/Dow: +153.10pts+1.4%/Dólar,12.37/ Mercados al alza favorecidos por dato positivo de la Balanza comercial en EU //ER
13851638767 2010-05-12 08:41:09 CDT <BMVMercados> Bolsa: +198pts+0.62%/Dow

13368230137 2010-05-04 09:50:33 CDT <BMVMercados> Foto del Mexder (Bolsa de Derivados dentro del Piso de Remates)  http://tweetphoto.com/21046896
13365811342 2010-05-04 08:56:05 CDT <BMVMercados> Bolsa: -441pts:-1.3%/Dow: -155pts:-1.4%/Dólar:12.40//Mercados en espera de reforma en Wall Street, condiciones mas severas para bancos//ER
13364509605 2010-05-04 08:26:12 CDT <BMVMercados> Imagen del campanazo de Chedraui  http://tweetphoto.com/21040123
13324902605 2010-05-03 15:41:43 CDT <BMVMercados> Bolsa/+145.13pts/ +0.44%/Vol:283.4m/Imp:$8,714m/Dólar:12.25/Dow+143.22pts/+1.30%/Datos pos EU / IMEF MANUFACT+//ER  http://myloc.me/6GfEO
13321393805 2010-05-03 14:18:14 CDT <BMVMercados> Personal de Control operativo...2:17 pm  http://tweetphoto.com/20958071
13320771719 2010-05-03 14:03:28 CDT <BMVMercados> Bolsa/+150.68pts/ +0.46%/Vol:223.7m/Imp:$6,871m/Dólar:12.23/Dow+162.26pts/+1.47%/Datos+ EU / Apoyo a Grecia//ER
13310967557 2010-05-03 10:25:55 CDT <BMVMercados> Mañana soleada en el Piso de